In [ ]:
import pandas as pd\n
from sklearn.model_selection import train_test_split\n,
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder\n,
import numpy as np\n,
from statistics import mode\n,
import jellyfish\n,
from fuzzywuzzy import fuzz\n,

In [ ]:
# import excel file and separate df by car make\n",
    "xls = pd.ExcelFile('../Resources/Car_Data.xlsx')\n",
    "df1 = pd.read_excel(xls, 'Volkswagon')\n",
    "df2 = pd.read_excel(xls, 'Toyota')\n",
    "df3 = pd.read_excel(xls, 'Porsche')\n",
    "df4 = pd.read_excel(xls, 'Mercedes')\n",
    "df5 = pd.read_excel(xls, 'Maserati')\n",
    "df6 = pd.read_excel(xls, 'Lincoln')\n",
    "df7 = pd.read_excel(xls, 'Lexus')\n",
    "# df8 = pd.read_excel(xls, 'Kia')\n",
    "# df9 = pd.read_excel(xls, 'Landrover')\n",
    "# df10 = pd.read_excel(xls, 'Volkswagon')\n",
    "# df11 = pd.read_excel(xls, 'Honda')\n",
    "# df12 = pd.read_excel(xls, 'Hyundai')\n",
    "# df13 = pd.read_excel(xls, 'Acura')\n",
    "# df14 = pd.read_excel(xls, 'Audi')\n",
    "# df15 = pd.read_excel(xls, 'BMW')\n"

In [ ]:
def clean_data(df):\n",
    "    # drop null values\n",
    "    new_df = df.dropna(how='any')\n",
    "    # drop duplicates\n",
    "    new_df = new_df.drop_duplicates()\n",
    "    # reformat datetime to date_time object\n",
    "    new_df['formatted_sold_date'] = pd.to_datetime(new_df['sold_date'], format='%Y-%m-%d-%H.%M.%S.%f')\n",
    "    new_df['sold_date_year'] = new_df['formatted_sold_date'].dt.year\n",
    "    new_df['sold_date_month'] = new_df['formatted_sold_date'].dt.month\n",
    "\n",
    "    # drop unneeded columns (dc = drop columns)\n",
    "    dc_df = new_df.drop(columns=['sold_date', 'formatted_sold_date', 'subseries', 'make', 'transmission', 'seller'])\n",
    "\n",
    "    # replace condition grade text values with numbers\n",
    "    dc_df['condition_grade'] = dc_df['condition_grade'].replace(\n",
    "    {'SL': 0, 'RG': 20, 'PR': 10, 'EC':50, 'CL':40, 'C': 40, 'AV':30, 'A': 30})\n",
    "########################## convert colors ##########################\n",
    "    # reorganize columns\n",
    "    # new_df = dc_df[['auction_code', 'car_year', 'model_group', 'color_group', 'body', 'mileage', 'condition_grade', 'times_run', 'sale_price', 'sold_date_year', 'sold_date_month' ]]\n",
    "\n",
    "    # # copy dataframe\n",
    "    # copy = new_df.copy(deep=False)\n",
    "\n",
    "    # return new_df\n",
    "    return dc_df\n",
    "\n",
    "# def binning_function(col, cut_points, labels=None):\n",
    "#     minvalue = col.min()\n",
    "#     maxvalue = col.max()\n",
    "#     break_points = [minvalue] + cut_points + [maxvalue]\n",
    "#     print(break_points)\n",
    "#     if not labels:\n",
    "#         labels = range(len(cut_points) + 1)\n",
    "#     col_bin = pd.cut(col, bins=break_points, labels=labels, include_lowest=True)\n",
    "#     return col_bin\n",
    "#\n",
    "#\n",
    "# cut_points = [25000, 50000, 75000, 100000]\n",
    "# labels = ['< 25k', '25k - 50k', '50k - 75k', '75k - 100k', '> 100k']\n",
    "# new_df['vehicle_mileage_category'] = binning_function(new_df['mileage'], cut_points, labels)\n",
    "# new_df"

In [ ]:
# # for each unique value in Description\n",
    "# def make_models(df):\n",
    "#     for d in df.model.unique():\n",
    "#         # df[d]\n",
    "#         # convert to string\n",
    "#     # compute Levensthein distance\n",
    "#     # and set to True if >= a 80\n",
    "#     # (you may have to play around with it)\n",
    "#         df[d] = df['model'].apply(\n",
    "#             lambda x : fuzz.ratio(x, d) >= 70\n",
    "#                 )\n",
    "#     # set a name for the group\n",
    "#     # here, simply the shortest\n",
    "#     m = np.min(df[df[d]==True].model)\n",
    "#     # assign the group\n",
    "#     df.loc[df.model==d, 'model_group'] = m\n"

In [ ]:
def create_colors(df):\n",
    "#     # for each unique value in Description\n",
    "#     for i in df.color:\n",
    "#         # compute Levensthein distance\n",
    "#         # and set to True if >= a 40\n",
    "#         # (you may have to play around with it)\n",
    "#         df[i] = df['color'].apply(\n",
    "#             lambda x : fuzz.ratio(x, i) >= 30\n",
    "#             )\n",
    "#     # set a name for the group\n",
    "#     # here, simply the shortest\n",
    "#     m2 = np.min(df[df[i]==True].color)\n",
    "#     # assign the group\n",
    "#     df.loc[df.color==i, 'color_group'] = m2\n",
    "#\n",
    "#     return df

In [ ]:
def create_other_color(df):\n",
    "#     # Choose a cutoff value and create a list of application types to be replaced\n",
    "#     # use the variable name `application_types_to_replace`\n",
    "#     # group colors\n",
    "#     color_cnt = df['color'].value_counts()\n",
    "#     color_types_to_replace = list(color_cnt[color_cnt < 50].index)\n",
    "#\n",
    "#     # Replace in dataframe\n",
    "#     for color in color_types_to_replace:\n",
    "#         # convert color to string\n",
    "#         df['color'] = df['color'].replace(color,\"Other\")\n",
    "#         df['color'] = df['color'].apply(str)\n",
    "#\n",
    "#     return df

In [ ]:
# def create_other_model(df):\n",
    "#      # Group cars models less than 100 into 'other' category\n",
    "#     # Choose a cutoff value and create a list of application types to be replaced\n",
    "#     # use the variable name `application_types_to_replace`\n",
    "#     model_cnt = df[\"model\"].value_counts()\n",
    "#     model_types_to_replace = list(model_cnt[model_cnt < 100].index)\n",
    "#\n",
    "#     # Replace in dataframe\n",
    "#     for model in model_types_to_replace:\n",
    "#         df['model'] = df['model'].replace(model,\"Other\")\n",
    "#         df['model'] = df['model'].apply(str)\n",
    "#\n",
    "#\n",
    "#     return df"